In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col, when

In [2]:
# Create a Spark session
spark = (SparkSession
         .builder
         .appName("Coffee Survey")
         .getOrCreate()
        )

In [3]:
coffee_file_path = "/home/jovyan/data_science_challenge/DataEngineerTakeHomeChallenge/projects/datasets/coffee_survey.csv"

# Read data from a CSV file
coffee_df = spark.read.option("header", "true").csv(coffee_file_path)

In [4]:
# Show Coffee Survey DataFrame
coffee_df.show(n=2, truncate=False)

+-------------+---------------+----+-----------+-------------------------------------------+----------+--------+--------------+-------------------+----------------+-----------------------------------+---------------+-----+---------+-------+--------+-----------+--------+---------+-------------------+----------------+----------------------------+--------------+-------------------+----------------+----------------------------+--------------+-------------------+----------------+----------------------------+--------------+-------------------+----------------+----------------------------+--------------+----------+---------+--------------+---+-----------+---------+---------------+-----+-----------+---------+------------+----------+---------------+---------------+------+--------------+---------------+--------------+----------------------+-----------------+---------------+---------------------+
|submission_id|age            |cups|where_drink|brew                                       |brew_othe

In [5]:
# Print schema of dataFrame
coffee_df.printSchema()

root
 |-- submission_id: string (nullable = true)
 |-- age: string (nullable = true)
 |-- cups: string (nullable = true)
 |-- where_drink: string (nullable = true)
 |-- brew: string (nullable = true)
 |-- brew_other: string (nullable = true)
 |-- purchase: string (nullable = true)
 |-- purchase_other: string (nullable = true)
 |-- favorite: string (nullable = true)
 |-- favorite_specify: string (nullable = true)
 |-- additions: string (nullable = true)
 |-- additions_other: string (nullable = true)
 |-- dairy: string (nullable = true)
 |-- sweetener: string (nullable = true)
 |-- style: string (nullable = true)
 |-- strength: string (nullable = true)
 |-- roast_level: string (nullable = true)
 |-- caffeine: string (nullable = true)
 |-- expertise: string (nullable = true)
 |-- coffee_a_bitterness: string (nullable = true)
 |-- coffee_a_acidity: string (nullable = true)
 |-- coffee_a_personal_preference: string (nullable = true)
 |-- coffee_a_notes: string (nullable = true)
 |-- coffee_

In [6]:
# Count the number of rows in the DataFrame
coffee_df.count()

4776

In [12]:
# List of columns to drop from the DataFrame
columns_to_drop = ["brew_other", "purchase", "purchase_other", "favorite_specify", "additions", "additions_other", "dairy", "coffee_a_bitterness", "coffee_a_acidity", 
                     "coffee_a_personal_preference", "coffee_a_notes", "coffee_b_bitterness", "coffee_b_acidity", "coffee_b_personal_preference", 
                     "coffee_b_notes", "coffee_c_bitterness", "coffee_c_acidity", "coffee_c_personal_preference", "coffee_c_notes", 
                     "coffee_d_bitterness", "coffee_d_acidity", "coffee_d_personal_preference", "coffee_d_notes", "prefer_overall", 
                     "why_drink_other", "know_source", "spent_equipment", "value_equipment", "gender_specify", "education_level", 
                     "ethnicity_race", "ethnicity_race_specify", "political_affiliation"]

# Drop the specified columns from the DataFrame
coffee_df = coffee_df.drop(*columns_to_drop)

# Drop duplicate rows from the DataFrame
coffee_df = coffee_df.dropDuplicates()

# Drop rows with any missing values
coffee_df = coffee_df.dropna()

# Remove non-numeric characters using regular expression
coffee_df = (coffee_df.withColumn("age", regexp_replace(col("age"), "years old", ""))
                      .withColumn("total_spend", regexp_replace(col("total_spend"), "\\$", ""))
                      .withColumn("most_paid", regexp_replace(col("most_paid"), "\\$", ""))
                      .withColumn("most_willing", regexp_replace(col("most_willing"), "\\$", "")))

# Show only first 5 rows of the DataFrame after droping specific columns
coffee_df.show(n=5, truncate=False)

+-------------+------+----+-----------------------------+---------------------------------------------------------+-------------------+---------+-----------+---------------+-----------+-------------+---------+----------+---------+--------------------------+-----------+--------------------------------------------------------------------------------------+-----+---------+------------+----------+------+------------------+---------------+
|submission_id|age   |cups|where_drink                  |brew                                                     |favorite           |sweetener|style      |strength       |roast_level|caffeine     |expertise|prefer_abc|prefer_ad|wfh                       |total_spend|why_drink                                                                             |taste|most_paid|most_willing|value_cafe|gender|employment_status |number_children|
+-------------+------+----+-----------------------------+---------------------------------------------------------+-------

In [16]:
# Convert 'cups' column to numeric values
# Other values are cast to integers
coffee_df = coffee_df.withColumn("cups",
                 when(coffee_df["cups"] == "Less than 1", 0)
                 .when(coffee_df["cups"] == "More than 4", 5)
                 .otherwise(coffee_df["cups"].cast("int")))

# Show 'cups' column after converting to numeric values
coffee_df.select("cups").show(10)

+----+
|cups|
+----+
|   2|
|   1|
|   4|
|   1|
|   2|
|   1|
|   1|
|   1|
|   0|
|   2|
+----+
only showing top 10 rows



In [17]:
# List of non-numeric values that should be converted to 0 in 'total_spend' column
non_numeric_values = [
    "Really disliked this one! Tasted like overripe, starting to ferment apples. Doesn't taste like coffee. Too sweet, too creamy, unpleasant texture.",
    "NA",
    "Coffee A",
    " went through a phase when cooling where it reminded me of crunchberries from the serial? strawberry flavored things",
    "I primarily work from home",
    "I do a mix of both",
    "Fruit, Natural, Blackberry?, Blueberry, well balanced",
    "Coffee D",
    "I primarily work in person",
    "Coffee B",
    "Coffee C"
]

# Convert non-numeric values in 'total_spend' column to '0'
# Convert '>100' in 'total_spend' column to '100-120'
# Convert '<20' in 'total_spend' column to '10-20'
coffee_df = coffee_df.withColumn("total_spend", 
                                  when(col("total_spend").isin(non_numeric_values), 0)
                                 .when(col("total_spend") == ">100", "100-120")
                                 .when(col("total_spend") == "<20", "10-20")
                                 .otherwise(col("total_spend")))

# Show 'total_spend' column after removing non-numeric values
coffee_df.select("total_spend").distinct().show(truncate=False)

+-----------+
|total_spend|
+-----------+
|20-40      |
|0          |
|100-120    |
|60-80      |
|80-100     |
|10-20      |
|40-60      |
+-----------+



In [18]:
# List of non_wfh values that should be converted to NA in 'wfh' column
non_wfh = [
    "Coffee A",
    "Coffee D",
    "Coffee B",
    "1",
    "4",
    "Coffee C",
    " and I knew which one would like it and hate it the second I tasted. Love the strong berry notes to it"
]

# Convert non_wfh values in 'wfh' column to 'NA'
coffee_df = coffee_df.withColumn("wfh", 
                                  when(col("wfh").isin(non_wfh), "NA")
                                 .otherwise(col("wfh")))   

# Show 'wfh' column after removing unrelated values
coffee_df.select("wfh").distinct().show(n = 10, truncate=False)

+--------------------------+
|wfh                       |
+--------------------------+
|NA                        |
|I primarily work from home|
|I do a mix of both        |
|I primarily work in person|
+--------------------------+



In [19]:
# Convert 'Coffee A' values in 'why_drink' column to 'NA'
coffee_df = coffee_df.withColumn("why_drink",
                                 when(col("why_drink") == "Coffee A", "NA")
                                 .when(col("why_drink") == "$40-$60", "NA")
                                 .otherwise(col("why_drink")))

# Show 'why_drink' column after removing 'Coffee A' values
coffee_df.select("why_drink").distinct().show(truncate=False)

+---------------------------------------------------------------------------------------------+
|why_drink                                                                                    |
+---------------------------------------------------------------------------------------------+
|It tastes good, I need the caffeine, I need the ritual, It makes me go to the bathroom       |
|It tastes good, I need the caffeine                                                          |
|It makes me go to the bathroom                                                               |
|It tastes good, I need the caffeine, It makes me go to the bathroom, I need the ritual, Other|
|It makes me go to the bathroom, I need the ritual, I need the caffeine, It tastes good       |
|I need the caffeine, Other                                                                   |
|I need the caffeine, I need the ritual                                                       |
|It tastes good, It makes me go to the b

In [20]:
# List of values to be coverted to 'Yes'
yes_values = [
    "It tastes good, I need the caffeine, I need the ritual, It makes me go to the bathroom",
    "It tastes good, I need the caffeine",
    "Coffee A",
    "It tastes good",
    "Coffee D",
    "It tastes good, I need the ritual, Other",
    "It tastes good, I need the ritual, It makes me go to the bathroom, I need the caffeine",
    "It tastes good, I need the caffeine, I need the ritual",
    "Coffee C",
    "It tastes good, I need the ritual"
]

# List of values to be coverted to 'No'
no_value =[
    "$20-$40",
    "$80-$100",
    "$40-$60"
]

# Convert related values in 'taste' column to 'Yes'
# Convert non-related values in 'taste' column to 'No'  
coffee_df = coffee_df.withColumn("taste", 
                                  when(col("taste").isin(yes_values), "Yes")
                                 .when(col("taste").isin(no_value), "No")
                                 .otherwise(col("taste")))     

# Show 'taste' column after converting specific values respective 'Yes' & 'No'
coffee_df.select("taste").distinct().show(truncate=False)

+-----+
|taste|
+-----+
|NA   |
|No   |
|Yes  |
+-----+

